In [1]:
!docker run -p 9000:9000 -p 9001:9001 \
  -e MINIO_ROOT_USER=minio \
  -e MINIO_ROOT_PASSWORD=minio123 \
  -v $(pwd)/minio_data:/data \
  minio/minio server /data --console-address ":9001"

docker: Error response from daemon: create $(pwd)/minio_data: "$(pwd)/minio_data" includes invalid characters for a local volume name, only "[a-zA-Z0-9][a-zA-Z0-9_.-]" are allowed. If you intended to pass a host directory, use absolute path

Run 'docker run --help' for more information


In [2]:
!pip install minio

from minio import Minio
from pathlib import Path


MINIO_ENDPOINT = "127.0.0.1:9000"   
MINIO_USER     = "minio"
MINIO_PASS     = "minio123"
BUCKET         = "model-registry"

client = Minio(MINIO_ENDPOINT, access_key=MINIO_USER, secret_key=MINIO_PASS, secure=False)

if not client.bucket_exists(BUCKET):
    client.make_bucket(BUCKET)

root = Path("model_registry")  
uploaded = []
for p in root.rglob("*"):
    if p.is_file():
        obj = p.relative_to(root).as_posix()   
        client.fput_object(BUCKET, obj, str(p))
        uploaded.append(obj)

print(f"Uploaded {len(uploaded)} objects.")
uploaded[:10]  # show a few


Uploaded 8 objects.


['model1/model.joblib',
 'model1/model.pt',
 'model2/model.joblib',
 'model2/model.pt',
 'model3/model.joblib',
 'model3/model.pt',
 'model4/model.joblib',
 'model4/model.pt']

In [3]:
!cd server
!docker build -t id_card:latest .

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 2B done
#1 DONE 0.0s
ERROR: failed to build: failed to solve: failed to read dockerfile: open Dockerfile: no such file or directory


In [4]:
!cd router
!docker build -t router:latest .

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 DONE 0.0s

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 2B done
#1 DONE 0.0s
ERROR: failed to build: failed to solve: failed to read dockerfile: open Dockerfile: no such file or directory


In [5]:
!docker run -d --name model2_a -p 50051:50051 \
  -e VARIANT=model2 \
  -e MINIO_ENDPOINT=host.docker.internal:9000 \
  -e MINIO_USER=minio -e MINIO_PASS=minio123 \
  -e REGISTRY_BUCKET=model-registry \
  id_card:latest


!docker run -d --name model4_a -p 50052:50051 \
  -e VARIANT=model4 \
  -e MINIO_ENDPOINT=host.docker.internal:9000 \
  -e MINIO_USER=minio -e MINIO_PASS=minio123 \
  -e REGISTRY_BUCKET=model-registry \
  id_card:latest

!docker run -d --name model2_b -p 50061:50051 \
  -e VARIANT=model2 \
  -e MINIO_ENDPOINT=host.docker.internal:9000 \
  -e MINIO_USER=minio -e MINIO_PASS=minio123 \
  -e REGISTRY_BUCKET=model-registry \
  id_card:latest


docker: Error response from daemon: Conflict. The container name "/model2_a" is already in use by container "d280fb7b5873f7f6bf97d0d95a322e909ca000c8b3038c563bb9b7c242323fd6". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information
docker: Error response from daemon: Conflict. The container name "/model4_a" is already in use by container "9d5784f2960d0504cb54ce7e2b828a9a9a6b6f101725c7f33159a3c4ad510cbe". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information
docker: Error response from daemon: Conflict. The container name "/model2_b" is already in use by container "23191a393d9cdbb1f687033bf976f524ba5599f00bfa8bb490fbab6e9dd52f2c". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


In [6]:
!docker run -d --name router -p 50050:50050 \
  -e BACKENDS="model2=host.docker.internal:50051;host.docker.internal:50061,model4=host.docker.internal:50052" \
  router:latest


docker: Error response from daemon: Conflict. The container name "/router" is already in use by container "5f736b320dcb15e472df928ad3211dbe98424ad4e6370fade617c2fa6416579f". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


In [7]:
import grpc
import cv2
import numpy as np

# ensure we can import the stubs
import sys
sys.path.append("server")

import inference_pb2 as pb
import inference_pb2_grpc as pbg

ROUTER_ADDR = "localhost:50050"

# open GRPC channel to router
channel = grpc.insecure_channel(ROUTER_ADDR)
stub = pbg.InferenceStub(channel)

def predict_image(img_path):
    # read the image
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Image not found: {img_path}")

    # encode to jpeg bytes
    ok, enc = cv2.imencode(".jpg", img)
    img_bytes = enc.tobytes()

    # prepare request
    req = pb.PredictRequest(image=img_bytes)

    # call using future to capture metadata (which worker served)
    call = stub.Predict.future(req)
    resp = call.result()
    trailers = {k: v for k, v in call.trailing_metadata()}

    print("Request:", img_path)
    print("Prediction:", resp.y)
    print(f"Inference time: {resp.infer_ms:.2f} ms")
    print("Served by model:", trailers.get("x-variant"))
    print("Backend endpoint:", trailers.get("x-target"))

    return resp


In [8]:
predict_image("sample.jpg")

_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:50050: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:50050: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.\r\n -- 10061)"}"
>

In [ ]:
predict_image("sample.jpg")

As we can see above that it is using different model which is selected randomly at this point. 